# RoPE Position Calculation in Spatial-MLLM

This notebook demonstrates how the model calculates `position_ids` during:

1. **Pre-fill stage** (first forward pass, `cache_position[0] == 0`)
2. **Generation stage** (subsequent forward passes, using cached `rope_deltas`)

## Key Concepts

- **RoPE (Rotary Position Embedding)**: A way to encode position information
- **rope_deltas**: Cached offset values from vision tokens to apply to text tokens
- **cache_position**: Current position in the generation sequence
- **position_ids**: 3D tensor `[3, batch_size, seq_length]` for temporal/height/width dimensions

In [ ]:
import torch

## Stage 1: Pre-fill Function

During pre-fill, we calculate `position_ids` from scratch and cache `rope_deltas`.

In [ ]:
def simulate_prefill_stage(
    input_ids,
    image_grid_thw,
    video_grid_thw,
    second_per_grid_ts,
    attention_mask,
    inputs_embeds,
):
    """
    Stage 1: Pre-fill (first forward pass)
    
    During pre-fill, we calculate position_ids from scratch and cache rope_deltas.
    
    Args:
        input_ids: [batch_size, seq_length] - token IDs
        image_grid_thw: [num_images, 3] - temporal/height/width grid for images
        video_grid_thw: [num_videos, 3] - temporal/height/width grid for videos
        second_per_grid_ts: [num_videos] - time interval per grid
        attention_mask: [batch_size, seq_length] - attention mask (1=attend, 0=ignore)
        inputs_embeds: [batch_size, seq_length, hidden_dim] - input embeddings
    
    Returns:
        position_ids: [3, batch_size, seq_length] - 3D position IDs (temporal, height, width)
        rope_deltas: [batch_size] - cached position offsets for future generations
    """
    print("\n" + "="*80)
    print("STAGE 1: PRE-FILL (Initial Forward Pass)")
    print("="*80)
    
    # This would normally call self.get_rope_index()
    # For demonstration, we'll create dummy outputs with realistic shapes
    
    batch_size, seq_length = input_ids.shape
    
    print(f"\nInput dimensions:")
    print(f"  input_ids: {input_ids.shape}")
    print(f"  inputs_embeds: {inputs_embeds.shape}")
    print(f"  attention_mask: {attention_mask.shape}")
    
    # Simulate get_rope_index output
    # In reality, this would calculate based on vision token positions
    # For vision tokens: uses grid_thw to compute 3D positions
    # For text tokens: sequential positions starting after vision tokens
    
    # Example: Let's say we have 12 vision tokens and 5 text tokens
    # Vision tokens get 3D positions based on grid layout
    # Text tokens get sequential positions starting from max_vision_pos + 1
    
    position_ids = torch.zeros(3, batch_size, seq_length, dtype=torch.long)
    
    # Dimension 0: temporal positions
    # Dimension 1: height positions  
    # Dimension 2: width positions
    
    # Example for first sample (simplified):
    # Vision part (first 12 tokens): 3D grid positions
    position_ids[0, 0, :12] = torch.tensor([0, 0, 0, 0, 50, 50, 50, 50, 100, 100, 100, 100])  # temporal
    position_ids[1, 0, :12] = torch.tensor([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1])  # height
    position_ids[2, 0, :12] = torch.tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])  # width
    
    # Text part (last 5 tokens): all dimensions get same sequential values
    position_ids[0, 0, 12:] = torch.arange(101, 106)  # temporal
    position_ids[1, 0, 12:] = torch.arange(101, 106)  # height
    position_ids[2, 0, 12:] = torch.arange(101, 106)  # width
    
    print(f"\nCalculated position_ids: {position_ids.shape}")
    print(f"  Shape: [3, {batch_size}, {seq_length}]")
    print(f"  Temporal dimension (dim 0): {position_ids[0, 0]}")
    print(f"  Height dimension (dim 1): {position_ids[1, 0]}")
    print(f"  Width dimension (dim 2): {position_ids[2, 0]}")
    
    # rope_deltas: the position offset where text starts (max vision pos + 1)
    # This gets cached for use in generation stage
    rope_deltas = torch.tensor([100], dtype=torch.long)  # max vision position was 100
    
    print(f"\nCached rope_deltas: {rope_deltas}")
    print(f"  Shape: {rope_deltas.shape}")
    print(f"  This represents the max position from vision tokens")
    print(f"  Text tokens will continue from this position + cache_position")
    
    return position_ids, rope_deltas

## Stage 2: Generation Function

During generation, we reuse cached `rope_deltas` to calculate `position_ids` efficiently without recalculating vision token positions.

### 🔑 Key Understanding: KV Cache

**Question 1**: Where does `inputs_embeds` come from during generation?
- **Answer**: Only the **newly generated token** (the last one generated)
- Shape: `[batch_size, 1, hidden_dim]` - note seq_length=1!

**Question 3**: Does `batch_size, seq_length, hidden_dim = inputs_embeds.shape` include all previous tokens from pre-fill?
- **Answer**: **NO!** Only the current new token!
- All previous tokens (from pre-fill stage) are stored in KV cache
- We don't re-process them - that's the efficiency of KV caching!

**Question 2**: What does `repeat_interleave` do?
- Handles batched generation where `batch_size` might differ from cached `rope_deltas` size
- In our simple demo: batch_size=1, so it's a no-op
- Real use case: If you cache for 1 sample but generate for 2 samples in parallel

**Flow Example**:
```
Pre-fill: Process tokens [0-16] → store keys/values in cache → cache rope_deltas
Gen step 1: Process ONLY token 17 → position_id = 17 + 100 = 117
Gen step 2: Process ONLY token 18 → position_id = 18 + 100 = 118
Gen step 3: Process ONLY token 19 → position_id = 19 + 100 = 119
```

In [ ]:
def simulate_generation_stage(
    cached_rope_deltas,
    cache_position,
    inputs_embeds,
):
    """
    Stage 2: Generation (subsequent forward passes during decoding)
    
    During generation, we reuse cached rope_deltas to calculate position_ids efficiently.
    We don't need to recalculate vision token positions.
    
    Args:
        cached_rope_deltas: [batch_size] - cached offset from pre-fill stage
        cache_position: [seq_length] - current positions in generation (e.g., [17], [18], ...)
        inputs_embeds: [batch_size, seq_length, hidden_dim] - ONLY THE NEW TOKEN! (seq_length=1)
                       All previous tokens are in KV cache, we don't reprocess them!
    
    Returns:
        position_ids: [3, batch_size, seq_length] - 3D position IDs for current tokens
    """
    print("\n" + "="*80)
    print("STAGE 2: GENERATION (Subsequent Forward Passes)")
    print("="*80)
    
    # Question 3: Does this include all previous tokens from pre-fill?
    # Answer: NO! Only the current new token (seq_length=1)
    batch_size, seq_length, hidden_dim = inputs_embeds.shape
    
    print(f"\nInput dimensions:")
    print(f"  inputs_embeds: {inputs_embeds.shape} (typically seq_length=1 during generation)")
    print(f"  ⚠️  This is ONLY the newly generated token, not all previous tokens!")
    print(f"  cache_position: {cache_position.shape} = {cache_position}")
    print(f"  cached_rope_deltas: {cached_rope_deltas.shape} = {cached_rope_deltas}")
    
    # Calculate delta: current position + rope offset
    delta = (cache_position[0] + cached_rope_deltas).to(inputs_embeds.device)
    print(f"\nCalculated delta: {delta}")
    print(f"  delta = cache_position[0] ({cache_position[0]}) + rope_deltas ({cached_rope_deltas[0]}) = {delta[0]}")
    
    # Create base position IDs for current sequence
    # For generation with seq_length=1, this is just [0]
    position_ids = torch.arange(seq_length, device=inputs_embeds.device)
    print(f"\nBase position_ids: {position_ids}")
    print(f"  This is just [0] since we're processing 1 token")
    
    # Expand to batch size: [batch_size, seq_length]
    position_ids = position_ids.view(1, -1).expand(batch_size, -1)
    print(f"\nExpanded to batch: {position_ids.shape} = {position_ids}")
    
    # Question 2: What does repeat_interleave do here?
    # Answer: Handle cases where batch_size > cached_rope_deltas.shape[0]
    # Example: If we cached deltas for 1 sample but now generate for 2 samples
    # In our demo: batch_size=1, delta.shape[0]=1, so this just keeps delta as-is
    delta = delta.repeat_interleave(batch_size // delta.shape[0], dim=0)
    print(f"\nDelta repeated for batch: {delta.shape} = {delta}")
    print(f"  (In this demo: 1 // 1 = 1, so no actual repetition happens)")
    
    # Add delta to get final positions
    # position_ids was [0], after adding delta it becomes [delta]
    position_ids = position_ids.add(delta)
    print(f"\nAfter adding delta: {position_ids.shape} = {position_ids}")
    print(f"  [0] + {delta[0].item()} = [{position_ids[0, 0].item()}]")
    
    # Expand to 3D (temporal, height, width dimensions all get same values)
    position_ids = position_ids.unsqueeze(0).expand(3, -1, -1)
    print(f"\nFinal position_ids: {position_ids.shape}")
    print(f"  Shape: [3, {batch_size}, {seq_length}]")
    print(f"  All dimensions: {position_ids[:, 0, 0]}")
    print(f"  This means: temporal={position_ids[0, 0, 0]}, height={position_ids[1, 0, 0]}, width={position_ids[2, 0, 0]}")
    
    return position_ids

### Visual Explanation

```
┌─────────────────────────────────────────────────────────────────┐
│                      PRE-FILL STAGE                             │
├─────────────────────────────────────────────────────────────────┤
│  Input: ALL tokens [0-16] (12 vision + 5 text)                 │
│  Process: inputs_embeds.shape = [1, 17, 3584]                  │
│  Output: position_ids for all 17 tokens                        │
│         Store K,V in cache for all tokens                      │
│         Cache rope_deltas = 100                                │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│                   GENERATION STEP 1                             │
├─────────────────────────────────────────────────────────────────┤
│  Input: ONLY new token [17]                                    │
│  Process: inputs_embeds.shape = [1, 1, 3584]  ← seq_len=1!    │
│  Compute: position_ids = 0 + (17 + 100) = 117                 │
│  KV Cache: Append this token's K,V to existing cache          │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│                   GENERATION STEP 2                             │
├─────────────────────────────────────────────────────────────────┤
│  Input: ONLY new token [18]                                    │
│  Process: inputs_embeds.shape = [1, 1, 3584]  ← seq_len=1!    │
│  Compute: position_ids = 0 + (18 + 100) = 118                 │
│  KV Cache: Append this token's K,V to existing cache          │
└─────────────────────────────────────────────────────────────────┘

Key Point: We NEVER reprocess tokens 0-16 after pre-fill!
           Their K,V are cached and reused!
```

## Setup: Create Fake Inputs

Let's create example inputs to demonstrate the workflow.

In [ ]:
batch_size = 1
prompt_seq_length = 17  # e.g., 12 vision tokens + 5 text tokens
hidden_dim = 3584

# Pre-fill stage inputs
input_ids = torch.randint(0, 1000, (batch_size, prompt_seq_length))
attention_mask = torch.ones(batch_size, prompt_seq_length)
inputs_embeds = torch.randn(batch_size, prompt_seq_length, hidden_dim)

# Vision grid info (example: video with 3 temporal patches, 2x2 spatial)
video_grid_thw = torch.tensor([[3, 2, 2]])  # [num_videos, 3]
image_grid_thw = None
second_per_grid_ts = torch.tensor([1.0])  # 1 second per temporal grid

print(f"Pre-fill inputs created:")
print(f"  batch_size: {batch_size}")
print(f"  prompt_seq_length: {prompt_seq_length}")
print(f"  hidden_dim: {hidden_dim}")
print(f"  video_grid_thw: {video_grid_thw} (temporal=3, height=2, width=2)")

## Run Stage 1: Pre-fill

This is the first forward pass where we calculate position IDs from scratch.

In [ ]:
cache_position = torch.tensor([0])  # First token position

# Condition check from the code
is_prefill = (cache_position is not None and cache_position[0] == 0)

print(f"Checking if we're in pre-fill stage:")
print(f"  cache_position: {cache_position}")
print(f"  cache_position[0] == 0: {is_prefill}")
print(f"  -> Will calculate position_ids from scratch")

position_ids_prefill, rope_deltas = simulate_prefill_stage(
    input_ids=input_ids,
    image_grid_thw=image_grid_thw,
    video_grid_thw=video_grid_thw,
    second_per_grid_ts=second_per_grid_ts,
    attention_mask=attention_mask,
    inputs_embeds=inputs_embeds,
)

## Run Stage 2: Generation (3 steps)

Now simulate token generation where we use the cached `rope_deltas`.

In [ ]:
for step in range(3):  # Simulate 3 generation steps
    # During generation, seq_length=1 (one token at a time)
    gen_seq_length = 1
    gen_inputs_embeds = torch.randn(batch_size, gen_seq_length, hidden_dim)
    gen_cache_position = torch.tensor([prompt_seq_length + step])
    
    print(f"\n{'─'*80}")
    print(f"Generation Step {step + 1}")
    print(f"{'─'*80}")
    print(f"Generating token at position: {gen_cache_position[0]}")
    
    position_ids_gen = simulate_generation_stage(
        cached_rope_deltas=rope_deltas,
        cache_position=gen_cache_position,
        inputs_embeds=gen_inputs_embeds,
    )

## Summary

### Key Insights:

**1. Pre-fill stage:**
- Calculates full 3D position_ids for vision tokens (temporal, height, width)
- Text tokens get sequential 1D positions (same across all 3 dims)
- Caches rope_deltas (max vision position) for later use

**2. Generation stage:**
- Uses cached rope_deltas to avoid recalculating vision positions
- `position_ids = cache_position[0] + rope_deltas`
- Much faster since it's just arithmetic, no grid calculations

**3. Why 3D position IDs?**
- **Temporal**: encodes time/frame information
- **Height**: encodes vertical spatial position
- **Width**: encodes horizontal spatial position
- Allows RoPE to understand spatial-temporal relationships in video/images

**4. Key dimensions:**
- `position_ids`: `[3, batch_size, seq_length]`
  - First dim (3): temporal, height, width
  - Used by RoPE to encode positional information
- `rope_deltas`: `[batch_size]`
  - Scalar offset per batch
  - Represents where text generation continues from